# DeepLabV3+ Versão Github

In [ ]:
https://github.com/topics/deeplabv3

In [ ]:
!git clone https://github.com/tensorflow/models.git
%cd models/research/deeplab

In [ ]:
!pip install -r requirements.txt
!pip install tf_slim
!pip install tensorflow==2.4.1
!pip install tf-models-official

In [ ]:
!protoc --python_out=. ./deeplab/protos/*.proto


In [ ]:
import sys
sys.path.append('/content/models')
sys.path.append('/content/models/research')
sys.path.append('/content/models/research/slim')


tratar dados

In [ ]:
!python train.py \
  --logtostderr \
  --training_number_of_steps=50000 \
  --train_split="train" \
  --model_variant="xception_65" \
  --atrous_rates=6 \
  --atrous_rates=12 \
  --atrous_rates=18 \
  --output_stride=16 \
  --decoder_output_stride=4 \
  --train_crop_size=513 \
  --train_crop_size=513 \
  --train_batch_size=1 \
  --dataset="dolphin" \
  --tf_initial_checkpoint="path_to_initial_trained_model/deeplabv3_pascal_train_aug/model.ckpt" \
  --train_logdir="path/to/training_log" \
  --dataset_dir="path/to/training_dataset" \
  --fine_tune_batch_norm=False

# Segunda implementação

In [ ]:
!pip install segmentation_models_pytorch

In [ ]:
import torch
import torch.optim as optim
import segmentation_models_pytorch as smp

# Definir o modelo DeepLabV3+ com ResNet34 como backbone
model = smp.DeepLabV3Plus(
    encoder_name='resnet34',
    encoder_depth=5,
    encoder_weights='imagenet',
    encoder_output_stride=16,
    decoder_channels=256,
    decoder_atrous_rates=(12, 24, 36),
    in_channels=3,
    classes=1,
    activation=None,
    upsampling=4,
    aux_params=None
)

# Configurar hiperparâmetros de treinamento
learning_rate = 0.001
momentum = 0.9
weight_decay = 1e-4
num_epochs = 50

# Definir o otimizador
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

# Definir a função de perda
criterion = torch.nn.CrossEntropyLoss()

# Configurar o DataLoader
from torch.utils.data import DataLoader
from torchvision.datasets import VOCSegmentation
from torchvision.transforms import Compose, ToTensor, Normalize

# Definir transformações
transform = Compose([
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Carregar o dataset
train_dataset = VOCSegmentation(root='./data', year='2012', image_set='train', download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)

# Função de treino
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, masks in loader:
        images, masks = images.to(device), masks.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, masks)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss

# Treinar o modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}')


In [ ]:
print(model)

# Terceira alternativa
https://github.com/VainF/DeepLabV3Plus-Pytorch

Construção do modelo usando o backbone pré treinado e construindo o deeplabv3+ do zero